# import data

In [1]:
import pandas as pd
import numpy as np

aseg_data = pd.read_csv('../dataset/final_data/aseg+DKT.stats980.csv')
aseg_data.head()

,Group,Age,Sex,Volume_mm30,normMean0,normStdDev0,normMin0,normMax0,Volume_mm31,normMean1,...,Volume_mm398,normMean98,normStdDev98,normMin98,normMax98,Volume_mm399,normMean99,normStdDev99,normMin99,normMax99
0,AD,78,M,239618.359,104.5263,8.7939,23.0,133.0,19019.868,21.8726,...,617.654,78.3675,12.5697,49.0,101.0,4935.454,70.7624,9.5176,42.0,95.0
1,AD,66,M,244462.620,104.4633,9.0946,19.0,135.0,52376.076,18.9785,...,866.656,81.6065,11.9081,49.0,106.0,5642.185,71.9476,9.2368,45.0,97.0
2,AD,77,M,236413.264,104.6093,8.0282,28.0,133.0,16591.926,27.7462,...,928.867,84.8252,10.9218,55.0,101.0,5959.229,76.6012,8.9234,50.0,99.0
3,AD,73,M,227601.449,104.3662,8.7423,29.0,132.0,19109.596,20.5128,...,576.647,80.3540,12.4069,51.0,105.0,5598.818,71.2126,8.7279,43.0,97.0
4,AD,62,M,220511.415,104.5355,7.3271,40.0,129.0,4690.350,36.5242,...,724.538,86.7110,10.1832,61.0,103.0,5327.887,76.5429,7.5031,51.0,96.0


In [2]:
aseg_X = aseg_data.iloc[:, 3:]
aseg_y = aseg_data.iloc[:, 0]
aseg_X = (aseg_X - aseg_X.mean()) / aseg_X.std()
aseg_X = aseg_X.fillna(0)
X, y = aseg_X.to_numpy(), aseg_y.to_numpy()

In [3]:
from sklearn.preprocessing import OneHotEncoder

target2idx = {'AD': 0, 'MCI': 1, 'CN': 2}
idx2target = {0: 'AD', 1: 'MCI', 2: 'CN'}
y_idx = np.array([target2idx[t] for t in y])
y_onehot = OneHotEncoder().fit_transform(y_idx.reshape(-1, 1)).toarray().astype(int)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

(
    X_train,
    X_test,
    y_train,
    y_test,
    y_train_onehot,
    y_test_onehot,
    y_train_idx,
    y_test_idx,
) = train_test_split(X, y, y_onehot, y_idx, test_size=0.2, random_state=42)

In [5]:
y_train.shape,  y_test.shape

((784,), (196,))

In [6]:
X_test_gnn = X_test.reshape(X_test.shape[0], -1, 5)
X_train_gnn = X_train.reshape(X_train.shape[0], -1, 5)


In [7]:
np.unique(y, return_counts=True)

(array(['AD', 'CN', 'MCI'], dtype=object), array([221, 282, 477]))

# pyg

## edge

In [8]:
def get_adjacency_matrix(X):
    # Compute the correlation matrix
    for i in range(X.shape[0]):
        if i == 0:
            corr_matrix = np.corrcoef(X[i])
        else:
            corr_matrix = np.dstack((corr_matrix, np.corrcoef(X[i])))
    # corr_matrix shape: [num_samples, num_nodes, num_nodes]
    return corr_matrix
# adjacency_matrix
adjacency_matrix = (get_adjacency_matrix(X_train_gnn).mean(axis=2) > 0.5).astype(int)
# 注意：要试一下去掉diagonal, self-looping
# adjacency_matrix = adjacency_matrix - np.eye(adjacency_matrix.shape[1])
# adjacency_matrix[adjacency_matrix == 0] = 1e-10
A_train_gnn = np.tile(adjacency_matrix, (X_train_gnn.shape[0], 1, 1))
A_test_gnn = np.tile(adjacency_matrix, (X_test_gnn.shape[0], 1, 1))
A_train_gnn, A_test_gnn = A_train_gnn.astype(int), A_test_gnn.astype(int)

/home/zqy/venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/zqy/venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [17]:
# adjacency_matrix = adjacency_matrix - np.eye(adjacency_matrix.shape[1])
# file = '../temp/adjacency_matrix.npy'
# print(adjacency_matrix.sum())
# np.save(file, adjacency_matrix)

## data

In [23]:
import torch
from torch_geometric.data import Data, Dataset


class CustomDataset(Dataset):
    def __init__(self, X, A, y):
        super(CustomDataset, self).__init__()
        self.X = X
        self.A = A
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float)
        edge_index = torch.tensor(self.A.nonzero(), dtype=torch.long)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return Data(x=x, edge_index=edge_index, y=y)



# Create dataset for training and testing
train_dataset = CustomDataset(X_train_gnn, adjacency_matrix, y_train_onehot)
test_dataset = CustomDataset(X_test_gnn, adjacency_matrix, y_test_onehot)

In [24]:
# train_dataset.get(0).edge_index.shape
train_dataset[0], test_dataset[0].edge_index

(Data(x=[100, 5], edge_index=[2, 1091], y=[3]),
 tensor([[ 0,  1,  1,  ..., 99, 99, 99],
         [ 0,  1,  9,  ..., 97, 98, 99]]))

In [25]:
train_dataset[0].x.dtype, train_dataset[0].edge_index.dtype, train_dataset[0].y.dtype

(torch.float32, torch.int64, torch.int64)

In [38]:
from torch.utils.data import DataLoader

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [39]:
for i in train_loader:
    print(i)
    break



TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torch_geometric.data.data.Data'>

## model

In [14]:
import torch
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.nn import TopKPooling

import torch.nn.functional as F
import torch_geometric.nn


class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, 32)
        self.pool_topk = TopKPooling(32, ratio=0.5)
        self.conv3 = GCNConv(32, num_classes)
        
        self.pool_gmp = torch_geometric.nn.global_mean_pool

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, _, batch, _, _ = self.pool_topk(x, edge_index, None, data.batch)
        x = self.conv3(x, edge_index)

        x = self.pool_gmp(x, batch)
        
        return F.log_softmax(x, dim=1)

# Example usage:
model = GCN(num_node_features=5, num_classes=3)
print(model)

GCN(
  (conv1): GCNConv(5, 16)
  (conv2): GCNConv(16, 32)
  (pool_topk): TopKPooling(32, ratio=0.5, multiplier=1.0)
  (conv3): GCNConv(32, 3)
)


In [15]:
import torch
from torch_geometric.nn import summary

print(summary(model, train_dataset[0]))

+-----------------------------+----------------------+--------------------------------------+----------+
| Layer                       | Input Shape          | Output Shape                         | #Param   |
|-----------------------------+----------------------+--------------------------------------+----------|
| GCN                         | [100, 100]           | [1, 3]                               | 771      |
| ├─(conv1)GCNConv            | [100, 5], [2, 1091]  | [100, 16]                            | 96       |
| ├─(conv2)GCNConv            | [100, 16], [2, 1091] | [100, 32]                            | 544      |
| ├─(pool_topk)TopKPooling    | [100, 32], [2, 1091] | [50, 32], [2, 137], [50], [50], [50] | 32       |
| │    └─(select)SelectTopK   | [100, 32], [100]     |                                      | 32       |
| │    │    └─(act)Tanh       | [100]                | [100]                                | --       |
| │    └─(connect)FilterEdges | [2, 1091], [100]     | 

In [22]:
for i in train_loader:
    print(i.batch, i)
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torch_geometric.data.data.Data'>

In [16]:
from torch_geometric.loader import DataLoader

import torch.optim as optim

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# Training function
def train():
    model.train()
    total_loss = 0
    for data in train_loader: ###
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader) ###

# Testing function
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

# Training loop
for epoch in range(1, 201):
    loss = train()
    train_acc = test(train_loader) ###
    test_acc = test(test_loader) ###
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torch_geometric.data.data.Data'>

# test

In [70]:
import torch
from torch_geometric.data import Data, DataLoader
import random

def create_graph(num_nodes, num_edges):
    edge_index = []
    for _ in range(num_edges):
        u = random.randint(0, num_nodes - 1)
        v = random.randint(0, num_nodes - 1)
        edge_index.append([u, v])
        edge_index.append([v, u])  # Undirected edges
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    x = torch.rand((num_nodes, 3))  # Node features
    y = torch.tensor([random.randint(0, 1)])  # Class label (binary)
    return Data(x=x, edge_index=edge_index, y=y)

# Create a list of synthetic graphs
graphs = [create_graph(num_nodes=10, num_edges=20) for _ in range(100)]

In [73]:

loader1 = DataLoader(graphs, batch_size=32, shuffle=True)


In [75]:
for i in loader1:
    print(i)
    break

DataBatch(x=[320, 3], edge_index=[2, 1280], y=[32], batch=[320], ptr=[33])


In [8]:
from torch_geometric.datasets import Planetoid

cora_dir = '../temp/Cora'
dataset = Planetoid(cora_dir, name="Cora")


In [12]:
from torch_geometric.datasets import TUDataset

dataset_mutag = TUDataset(root='../temp/MUTAG', name='MUTAG')

Processing...
Done!


In [16]:
from torch_geometric.loader import DataLoader

dataload_mutag = DataLoader(dataset_mutag, batch_size=32, shuffle=True)

In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn as nn

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)
        self.fc = nn.Linear(out_channels, 2)  # Assuming binary classification

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, data.batch)  # Global mean pooling
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

# Load the MUTAG dataset
dataset = TUDataset(root='../temp/MUTAG', name='MUTAG')
train_loader = DataLoader(dataset[:150], batch_size=32, shuffle=True)
test_loader = DataLoader(dataset[150:], batch_size=32)

# Create model, optimizer and loss function
model = GCN(in_channels=dataset.num_node_features, out_channels=16) # Change as per your dataset
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

# Training Loop
model.train()
for epoch in range(1):  # Number of epochs
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}: Loss={loss.item():.4f}')

# Testing Loop
model.eval()
correct = 0
for data in test_loader:
    out = model(data)
    preds = out.argmax(dim=1)
    correct += (preds == data.y).sum().item()

accuracy = correct / len(test_loader.dataset)
print(f'Test Accuracy: {accuracy:.4f}')

Epoch 1: Loss=0.6581
Test Accuracy: 0.6842


/home/zqy/venv/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [10]:
from torch_geometric.nn import summary

print(summary(model, dataset[0]))

+------------------+-------------------+----------------+----------+
| Layer            | Input Shape       | Output Shape   |   #Param |
|------------------+-------------------+----------------+----------|
| GCN              | [17, 17]          | [1, 2]         |      434 |
| ├─(conv1)GCNConv | [17, 7], [2, 38]  | [17, 16]       |      128 |
| ├─(conv2)GCNConv | [17, 16], [2, 38] | [17, 16]       |      272 |
| ├─(fc)Linear     | [1, 16]           | [1, 2]         |       34 |
+------------------+-------------------+----------------+----------+


In [11]:
for i in train_loader:
    # print(model(i).shape)
    print(i)
    break

DataBatch(edge_index=[2, 1240], x=[558, 7], edge_attr=[1240, 4], y=[32], batch=[558], ptr=[33])


In [13]:
dataset[0].x.dtype, dataset[0].edge_index.dtype, dataset[0].y.dtype


(torch.float32, torch.int64, torch.int64)